In [324]:
this_file_name = 'test_position_control_class.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))
from utils import *
import pid
from controller_utils import *
from controllers import *

# Connecting to the Simulator

In [325]:
clientID = connect_2_sim()
test_connection(clientID)

Connected to remote API server
Number of objects in the scene:  57


## Getting robot handle

In [326]:
p3dx_handle = get_pioneer3DX_handle(clientID)

PioneerP3DX handler successfully retrieved!


## Getting Initial Config

In [327]:
p3dx_position = sim.simxGetObjectPosition(clientID,p3dx_handle,-1,sim.simx_opmode_blocking)
p3dx_position[1][0:2]

[-0.22625264525413513, -0.44886064529418945]

In [328]:
p3dx_orientation = sim.simxGetObjectOrientation(clientID,p3dx_handle,-1,sim.simx_opmode_blocking)
p3dx_orientation[1][2]

-0.001524896826595068

In [329]:
p3dx_initial_config = np.concatenate((p3dx_position[1][0:2], [p3dx_orientation[1][2]])) # x,y,theta
p3dx_initial_config

array([-0.22625265, -0.44886065, -0.0015249 ])

## Getting Motor Handles

In [330]:
left_motor_handle, right_motor_handle = get_pioneer3DX_motor_handles_(clientID)

motor handles successfully retrieved!


## Testing Get Metaparameters

In [331]:
p1 = np.array([3,3,3.14])
p2 = np.array([1,1,1.57])

del_l,del_theta = get_metaparameters(p1,p2) # del_l = 2*sqrt(2)*cos(-1.57), del_theta = -1.57

delta_l = del_l/np.cos(del_theta)
delta_l/np.sqrt(2)

2.0

# A Trivial Situation: Same position, other orientation

## Controller Stuff

### Instancing

In [314]:
delL_params = np.array([0.9,0.01,0])
delTh_params = np.array([0.01,0.00001,0]) # apparently ok
uncoupling_m = np.array([[0.01,1.954],[0.01,-1.954]])

In [315]:
u_c = UncoupledController(delL_controller_params=delL_params,
                          delTH_controller_params=delTh_params,
                          uncoupling_matrix=uncoupling_m)

### Final Config

In [316]:
final_config = p3dx_initial_config.copy()
final_config[-1] = 1.57
final_config

array([-0.22240421, -0.44948477,  1.57      ])

In [317]:
p3dx_initial_config

array([-2.22404212e-01, -4.49484766e-01, -3.51456663e-04])

### Run

In [318]:
del_l, del_theta = get_metaparameters(final_config, p3dx_initial_config)
print(f'original del_l = {del_l}, original del_theta = {del_theta}')

original del_l = 0.0, original del_theta = -1.5703514566633385


In [319]:
while np.abs(del_l) > 0.1 or np.abs(del_theta) > 0.1:
    wheels_speed = u_c.control(del_l,del_theta)
    print(wheels_speed)
    return_value_left_motor_control, return_value_right_motor_control = robot_run(clientID, left_motor_handle, right_motor_handle, 
                                                                                    wheels_speed[0][0],wheels_speed[1][0])
    if return_value_left_motor_control != 0 or return_value_right_motor_control != 0:
        print(f'motor failure!!')
    #time.sleep()
    p3dx_position = sim.simxGetObjectPosition(clientID,p3dx_handle,-1,sim.simx_opmode_blocking)
    p3dx_orientation = sim.simxGetObjectOrientation(clientID,p3dx_handle,-1,sim.simx_opmode_blocking)
    p3dx_current_config = np.concatenate((p3dx_position[1][0:2], [p3dx_orientation[1][2]])) # x,y,theta
    del_l, del_theta = get_metaparameters(final_config, p3dx_current_config)

    print(f'current config = {p3dx_current_config}')
    print(f'del_l = {del_l}, del_theta = {del_theta}')
print('NOT RUNNING')
    

[[-0.03071535]
 [ 0.03071535]]
current config = [-0.22346011 -0.44934925  0.00134204]
del_l = 2.276410124608332e-06, del_theta = -1.568657960095443
[[-0.03071289]
 [ 0.03071293]]
current config = [-0.2234606  -0.44950286  0.0049459 ]
del_l = 6.066873454009438e-06, del_theta = -1.565054102037102
[[-0.03067302]
 [ 0.03067313]]
current config = [-0.22344494 -0.44967318  0.00855297]
del_l = 9.888087319097424e-06, del_theta = -1.5614470268413425
[[-0.03063301]
 [ 0.03063319]]
current config = [-0.22341709 -0.44983608  0.01216192]
del_l = 1.3891777101052738e-05, del_theta = -1.557838082946837
[[-0.0305929 ]
 [ 0.03059315]]
current config = [-0.22340195 -0.44999725  0.01578558]
del_l = 1.859837176201442e-05, del_theta = -1.5542144176363946
[[-0.03055241]
 [ 0.03055276]]
current config = [-0.22340338 -0.45015058  0.01939677]
del_l = 2.4243899444261e-05, del_theta = -1.550603225529194
[[-0.0305121 ]
 [ 0.03051255]]
current config = [-0.22336295 -0.45030719  0.02300764]
del_l = 3.006530943942063

# A Trivial Situation: Other position, same orientation

## Controller Stuff

### Instancing

In [332]:
delL_params = np.array([0.9,0.001,0])
delTh_params = np.array([0.01,0.00001,0])
uncoupling_m = np.array([[0.01,1.954],[0.01,-1.954]])

In [333]:
u_c = UncoupledController(delL_controller_params=delL_params,
                          delTH_controller_params=delTh_params,
                          uncoupling_matrix=uncoupling_m)

### Final Config

In [334]:
final_config = p3dx_initial_config.copy()
final_config[:2] = [1,0]
final_config

array([ 1.       ,  0.       , -0.0015249])

In [335]:
p3dx_initial_config

array([-0.22625265, -0.44886065, -0.0015249 ])

### Run

In [336]:
del_l, del_theta = get_metaparameters(final_config, p3dx_initial_config)
print(f'original del_l = {del_l}, original del_theta = {del_theta}')

original del_l = 1.305822127583493, original del_theta = 0.0


In [337]:
while np.abs(del_l) > 0.1 or np.abs(del_theta) > 0.1:
    wheels_speed = u_c.control(del_l,del_theta)
    print(wheels_speed)
    return_value_left_motor_control, return_value_right_motor_control = robot_run(clientID, left_motor_handle, right_motor_handle, 
                                                                                    wheels_speed[0][0],wheels_speed[1][0])
    if return_value_left_motor_control != 0 or return_value_right_motor_control != 0:
        print(f'motor failure!!')
    #time.sleep()
    p3dx_position = sim.simxGetObjectPosition(clientID,p3dx_handle,-1,sim.simx_opmode_blocking)
    p3dx_orientation = sim.simxGetObjectOrientation(clientID,p3dx_handle,-1,sim.simx_opmode_blocking)
    p3dx_current_config = np.concatenate((p3dx_position[1][0:2], [p3dx_orientation[1][2]])) # x,y,theta
    del_l, del_theta = get_metaparameters(final_config, p3dx_current_config)

    print(f'current config = {p3dx_current_config}')
    print(f'del_l = {del_l}, del_theta = {del_theta}')
print('NOT RUNNING')
    

[[0.01188298]
 [0.01188298]]
current config = [-0.23767789 -0.44595593 -0.00287688]
del_l = 1.3155683494480856, del_theta = -0.0013519828207790852
[[0.01207581]
 [0.0121287 ]]
current config = [-0.23750401 -0.44593725 -0.00286452]
del_l = 1.3153984513260928, del_theta = -0.001339627429842949
[[0.01220604]
 [0.01225849]]
current config = [-0.2372193  -0.44592682 -0.00285786]
del_l = 1.3151270879036723, del_theta = -0.001332959160208702
[[0.01233521]
 [0.01238746]]
current config = [-0.23710227 -0.44592661 -0.00284621]
del_l = 1.3150169376043666, del_theta = -0.0013213148340582848
[[0.01246592]
 [0.01251777]]
current config = [-0.23683968 -0.44592899 -0.00284186]
del_l = 1.3147707273031135, del_theta = -0.0013169676531106234
[[0.01259524]
 [0.01264697]]
current config = [-0.23661223 -0.44590554 -0.00283662]
del_l = 1.314548813404505, del_theta = -0.001311723841354251
[[0.01272478]
 [0.01277635]]
current config = [-0.23646002 -0.44590729 -0.00285761]
del_l = 1.3144061838300924, del_theta 

KeyboardInterrupt: 

# A Non Trivial Test

## Controller Stuff

### Instancing

In [140]:
delL_params = np.array([1.0,0.5,0])
delTh_params = np.array([0.1,0.001,0])
uncoupling_m = np.array([[0.01,1.954],[0.01,-1.954]])

In [141]:
u_c = UncoupledController(delL_controller_params=delL_params,
                          delTH_controller_params=delTh_params,
                          uncoupling_matrix=uncoupling_m)

### Final Config

In [142]:
final_config = np.array([1,1,1.57]) 

### Running the controller

In [143]:
del_l, del_theta = get_metaparameters(final_config, p3dx_initial_config)

while del_l > 0.1:
    wheels_speed = u_c.control(del_l,del_theta)
    print(wheels_speed)
    return_value_left_motor_control, return_value_right_motor_control = robot_run(clientID, left_motor_handle, right_motor_handle, 
                                                                                    wheels_speed[1][0], wheels_speed[0][0])
    if return_value_left_motor_control != 0 or return_value_right_motor_control != 0:
        print(f'motor failure!!')
    #time.sleep()
    p3dx_position = sim.simxGetObjectPosition(clientID,p3dx_handle,-1,sim.simx_opmode_blocking)
    p3dx_orientation = sim.simxGetObjectOrientation(clientID,p3dx_handle,-1,sim.simx_opmode_blocking)
    p3dx_current_config = np.concatenate((p3dx_position[1][0:2], [p3dx_orientation[1][2]])) # x,y,theta
    del_l, del_theta = get_metaparameters(final_config, p3dx_current_config)

    print(f'current config = {p3dx_current_config}')
    print(f'del_l = {del_l}, del_theta = {del_theta}')

    

[[0.01090509]
 [0.04599737]]
current config = [-0.22045843 -0.46252424  1.47953951]
del_l = 1.8970726094090509, del_theta = -0.0904604864120484
[[0.01991337]
 [0.0559663 ]]
current config = [-0.22049925 -0.46194911  1.47732139]
del_l = 1.8962726500355882, del_theta = -0.09267861366271979
[[0.02877222]
 [0.06605418]]
current config = [-0.22052138 -0.46120581  1.47506917]
del_l = 1.895317045257793, del_theta = -0.0949308347702027
[[0.03761367]
 [0.07614678]]
current config = [-0.22049668 -0.45996317  1.47274423]
del_l = 1.8939275555339476, del_theta = -0.09725577354431159
[[0.04642508]
 [0.08624686]]
current config = [-0.2204797  -0.45888433  1.47035706]
del_l = 1.8926467344582856, del_theta = -0.09964293956756598
[[0.05521435]
 [0.09635844]]
current config = [-0.22044678 -0.45755941  1.46791291]
del_l = 1.8911468620227454, del_theta = -0.10208708763122565
[[0.06397802]
 [0.10647624]]
current config = [-0.22040549 -0.45607066  1.46534967]
del_l = 1.8894827220157437, del_theta = -0.104650